# 🥈  SILVER LAYER

In [0]:
%run "./00 - DDL"

In [0]:
spark.sql(
    f"""
    USE SCHEMA {SILVER_SCHEMA}
    """
)

### GAMES

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {SILVER_GAMES_TABLE} (
    gamePk BIGINT COMMENT 'Unique game identifier assigned by the MLB Stats API.',
    game_date DATE COMMENT 'Official calendar date of the game.',
    season INT COMMENT 'MLB season year (e.g., 2024).',
    game_type STRING COMMENT 'Normalized game type: Regular Season, Postseason, Spring Training, or Exhibition.',
    home_team_name STRING COMMENT 'Full name of the home team.',
    away_team_name STRING COMMENT 'Full name of the visiting team.',
    venue_name STRING COMMENT 'Name of the ballpark or stadium.',
    home_score INT COMMENT 'Total number of runs scored by the home team.',
    away_score INT COMMENT 'Total number of runs scored by the away team.',
    home_is_winner BOOLEAN COMMENT 'True if the home team won the game.',
    away_is_winner BOOLEAN COMMENT 'True if the away team won the game.',
    attendance INT COMMENT 'Official attendance count for the game.',
    attendance_missing BOOLEAN COMMENT 'True if attendance is missing or reported as zero.',
    dayNight STRING COMMENT 'Indicates whether the game was played during the day or at night.',
    day_of_week STRING COMMENT 'Day of the week the game took place (e.g., Monday, Friday).',
    is_weekend BOOLEAN COMMENT 'True if the game was played on a Saturday or Sunday.',
    has_promotion BOOLEAN COMMENT 'True if the game featured any promotion, giveaway, or themed event.',
    num_promotions INT COMMENT 'Number of distinct promotions linked to this game.',
    offer_names ARRAY<STRING> COMMENT 'Array of promotional offer names tied to the game (e.g., "Bobblehead Giveaway").',
    promotion_types_array ARRAY<STRING> COMMENT 'Array of general promotion categories (e.g., "Giveaway", "Theme Game").',
    offer_types_array ARRAY<STRING> COMMENT 'Array of specific promotional type labels (e.g., "Day of Game Highlight").',
    seriesGameNumber INT COMMENT 'Index of the game within a multi-game series.',
    gamesInSeries INT COMMENT 'Total number of games scheduled in the series.',
    is_doubleheader_date BOOLEAN COMMENT 'True if multiple games were played between the same teams on this date.',
    doubleHeader STRING COMMENT 'Marks whether the game was part of a doubleheader (Y or N).',
    gameNumber BIGINT COMMENT 'Identifies which game in a doubleheader this record represents (1 or 2).',
    description STRING COMMENT 'Text description of the game (e.g., "home opener", "in Seoul, South Korea").'
)
COMMENT 'Silver-level cleaned version of all MLB games. Each record represents one game, with standardized fields and promotion arrays.';
""")

In [0]:
spark.sql(f"""
INSERT OVERWRITE {SILVER_GAMES_TABLE}
SELECT
    CAST(gamePk AS BIGINT) AS gamePk,
    CAST(date AS DATE) AS game_date,
    CAST(season AS INT) AS season,
    CASE gameType
        WHEN 'R' THEN 'Regular Season'
        WHEN 'P' THEN 'Postseason'
        WHEN 'S' THEN 'Spring Training'
        WHEN 'E' THEN 'Exhibition'
        ELSE 'Other'
    END AS game_type,
    home_team_name,
    away_team_name,
    venue_name,
    CAST(home_score AS INT) AS home_score,
    CAST(away_score AS INT) AS away_score,
    CAST(home_is_winner AS BOOLEAN) AS home_is_winner,
    CAST(away_is_winner AS BOOLEAN) AS away_is_winner,
    CAST(attendance AS INT) AS attendance,
    CASE WHEN attendance IS NULL OR attendance = 0 THEN TRUE ELSE FALSE END AS attendance_missing,
    LOWER(dayNight) AS dayNight,
    date_format(game_date, 'EEEE') AS day_of_week,
    CASE WHEN dayofweek(game_date) IN (1, 7) THEN TRUE ELSE FALSE END AS is_weekend,
    CAST(has_promotion AS BOOLEAN) AS has_promotion,
    COALESCE(num_promotions, size(offer_names)) AS num_promotions,
    COALESCE(offer_names, ARRAY()) AS offer_names,
    COALESCE(promotion_types_array, ARRAY()) AS promotion_types_array,
    COALESCE(offer_types_array, ARRAY()) AS offer_types_array,
    CAST(seriesGameNumber AS INT) AS seriesGameNumber,
    CAST(gamesInSeries AS INT) AS gamesInSeries,
    CAST(is_doubleheader_date AS BOOLEAN) AS is_doubleheader_date,
    doubleHeader,
    gameNumber,
    description
FROM {CATALOG}.{BRONZE_SCHEMA}.{BRONZE_GAMES_TABLE};"""
)

In [0]:
display(spark.sql(f"""
        DESCRIBE EXTENDED {CATALOG}.{SILVER_SCHEMA}.{SILVER_GAMES_TABLE};

"""))


In [0]:
# 1. Count of total games
total_games_df = spark.sql(f"""
    SELECT COUNT(*) AS total_games
    FROM {SILVER_GAMES_TABLE}
""")
display(total_games_df)

# 2. Count of games with promotions
promo_games_df = spark.sql(f"""
    SELECT 
        COUNT(*) AS total_games,
        COUNT_IF(has_promotion) AS games_with_promotions
    FROM {SILVER_GAMES_TABLE}
""")
display(promo_games_df)

# 3. Attendance distribution by season
attendance_dist_df = spark.sql(f"""
    SELECT 
        season, 
        ROUND(AVG(attendance),0) AS avg_attendance, 
        MAX(attendance) AS max_attendance, 
        MIN(attendance) AS min_attendance
    FROM {SILVER_GAMES_TABLE}
    GROUP BY season
""")
display(attendance_dist_df)

# 4. Sample of games with promotions
sample_promos_df = spark.sql(f"""
    SELECT 
        gamePk, 
        home_team_name, 
        offer_names, 
        promotion_types_array, 
        attendance
    FROM {SILVER_GAMES_TABLE}
    WHERE has_promotion = TRUE
    LIMIT 10
""")
display(sample_promos_df)


### Opponent Popularity: Average Attendance by Home Team

Compute the average attendance for every home team in each season, then join this metric to games where that team is the *away* team. This allows you to analyze how popular each opponent is, based on the crowds they draw on the road.

In [0]:
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW opponent_popularity AS
SELECT
  season,
  home_team_name AS opponent_name,
  ROUND(AVG(attendance), 0) AS opponent_avg_attendance
FROM {CATALOG}.{SILVER_SCHEMA}.{SILVER_GAMES_TABLE}
WHERE attendance IS NOT NULL AND attendance > 0
GROUP BY season, home_team_name;
          """)

Join that into your cleaned table:

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {CATALOG}.{SILVER_SCHEMA}.{SILVER_GAMES_TABLE_ENRICHED} AS
SELECT
  g.*,
  o.opponent_avg_attendance
FROM {CATALOG}.{SILVER_SCHEMA}.{SILVER_GAMES_TABLE} g
LEFT JOIN opponent_popularity o
  ON g.season = o.season
 AND g.away_team_name = o.opponent_name;""")



### Add home team performance (win percentage)
This gives you each team’s rolling win rate (updated per game). We’ll compute each team’s current win rate up to that game. You can use a window function for that:

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {CATALOG}.{SILVER_SCHEMA}.{SILVER_GAMES_TABLE_ENRICHED} AS
SELECT
  *,
  ROUND(
    SUM(CASE WHEN home_is_winner THEN 1 ELSE 0 END)
      OVER (PARTITION BY season, home_team_name ORDER BY game_date
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)/ COUNT(*) OVER (PARTITION BY season, home_team_name ORDER BY game_date
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW), 3
  ) AS home_team_win_pct
FROM {CATALOG}.{SILVER_SCHEMA}.{SILVER_GAMES_TABLE_ENRICHED};
          """
)


### Make sure is_weekend and dayNight are preserved

### Multiple Home Ballparks?

In [0]:
teams_multiple_venues_df = spark.sql(f"""
    SELECT 
        home_team_name,
        COUNT(DISTINCT venue_name) AS num_ballparks
    FROM {CATALOG}.{SILVER_SCHEMA}.{SILVER_GAMES_TABLE_ENRICHED}
    GROUP BY home_team_name
    HAVING COUNT(DISTINCT venue_name) > 1
    ORDER BY num_ballparks DESC
""")
display(teams_multiple_venues_df)

### Promotion type (already in your data)
When exploding promotion arrays, each game’s attendance remains *unchanged*—even if the game features multiple promotions.

> **Key Principle:**  
> - **1 game = 1 attendance**  
> - Exploding arrays provides analysis flexibility, but when aggregating, always use `COUNT(DISTINCT gamePk)` or `AVG(attendance)` per game—not `SUM(attendance)`.

This ensures accurate attendance metrics and avoids double-counting.

In [0]:
spark.sql(f"""
CREATE OR REPLACE VIEW {CATALOG}.{SILVER_SCHEMA}.{SILVER_PROMOTIONS_VIEW} AS
SELECT DISTINCT
    g.gamePk,
    g.season,
    g.home_team_name,
    g.away_team_name,
    g.venue_name,
    g.attendance,
    exploded.col.promotion_types_array AS promotion_type,
    exploded.col.offer_types_array AS offer_type,
    exploded.col.offer_names AS offer_name,
    g.day_of_week,
    g.is_weekend,
    g.dayNight,
    g.opponent_avg_attendance,
    g.home_team_win_pct
FROM {CATALOG}.{SILVER_SCHEMA}.{SILVER_GAMES_TABLE_ENRICHED} g
LATERAL VIEW OUTER EXPLODE(
    arrays_zip(
        g.promotion_types_array,
        g.offer_types_array,
        g.offer_names
    )
) exploded
WHERE g.has_promotion = TRUE
  -- Keep only valid promo rows (at least one value present)
  AND (
    exploded.col.promotion_types_array IS NOT NULL
    OR exploded.col.offer_types_array IS NOT NULL
    OR exploded.col.offer_names IS NOT NULL
  );
""")

## Quick Check

In [0]:
spark.sql(f"""
SELECT 
    promotion_type,
    offer_type,
    offer_name
FROM {CATALOG}.{SILVER_SCHEMA}.{SILVER_PROMOTIONS_VIEW}
WHERE promotion_type IS NULL 
AND offer_type IS NULL 
AND offer_name IS NULL
LIMIT 10;
""")

In [0]:
promo_type_null_offer_name_exists_df = spark.sql(f"""
    SELECT 
        gamePk,
        home_team_name,
        away_team_name,
        venue_name,
        offer_name,
        promotion_type
    FROM {CATALOG}.{SILVER_SCHEMA}.{SILVER_PROMOTIONS_VIEW}
    WHERE promotion_type IS NULL
      AND offer_name IS NOT NULL
    LIMIT 2000
""")
display(promo_type_null_offer_name_exists_df)

### Sanity Check

If that output looks realistic (i.e., teams with large ballparks like Dodgers, Yankees show higher averages), then you’re ready for Gold.

In [0]:
sample_promo_exploded_df = spark.sql(f"""
        SELECT 
        home_team_name,
        away_team_name,
        venue_name,
        COUNT(DISTINCT gamePk) AS num_games,
        COUNT(DISTINCT offer_name) AS num_promos,
        ROUND(AVG(attendance), 0) AS avg_attendance,
        ROUND(AVG(opponent_avg_attendance), 0) AS avg_opponent_attendance,
        ROUND(AVG(home_team_win_pct),3) AS avg_home_team_win_pct
        FROM {CATALOG}.{SILVER_SCHEMA}.{SILVER_PROMOTIONS_VIEW}
        GROUP BY home_team_name, away_team_name, venue_name
        ORDER BY avg_attendance DESC;
          """)
display(sample_promo_exploded_df)


## Verify Everything

At this point, your Silver table should now include:

| Column                  | Description                                               |
|-------------------------|----------------------------------------------------------|
| `is_weekend`            | Weekend vs weekday                                       |
| `dayNight`              | Day or night game                                        |
| `promotion_type`        | Type of promotion (exploded)                             |
| `opponent_avg_attendance` | Average attendance drawn by the visiting team           |
| `home_team_win_pct`     | Running win percentage for home team up to game day      |
| `attendance`            | Actual game attendance                                   |